In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib as mpl
import h5py
from collections import defaultdict
import operator as op
from matplotlib.collections import LineCollection
from scipy import sparse
import anndata
import random
from scipy.stats import gaussian_kde
import csv
import umap
from sklearn.preprocessing import StandardScaler


In [ ]:
# 3 Prime and Reverse Matching Dictionaries

matching_3p_10X_CBs = {} # Keys = GEX Capture Seq, Values = Spatial Barcode Capture Seq
reverse_matching_3p_10X_CBs = {} # Keys = Spatial Barcode Capture Seq, Values = GEX Capture Seq

# Go through all matching pairs
with open('3p_10X_CB_matching.txt', 'r') as f:
    for CB_pair in f.readlines():
        CB_1, CB_2 = CB_pair.split()
        
        # Add corresponding key, value pairs to matching dictionaries
        matching_3p_10X_CBs[CB_1] = CB_2
        reverse_matching_3p_10X_CBs[CB_2] = CB_1


In [ ]:
### GEX DATA OF ALL CELL BARCODES ###

# Load raw h5 file from CellRanger and add up GEX UMIs per CB
gel_2_GEX = sc.read_10x_h5('gel_2/CellRanger_gel_2/gel_2_raw_feature_bc_matrix.h5')
gel_2_GEX.obs['Total GEX UMIs'] = list(gel_2_GEX.X.sum(axis=1).squeeze().A1)
gel_2_GEX.obs.index = gel_2_GEX.obs.index.map(lambda CB: CB[:-2])

# Get total GEX UMIs
total_GEX_UMIs = sum(gel_2_GEX.obs['Total GEX UMIs'])

# Print GEX UMI metrics
print(f'Number of droplets analyzed: {len(gel_2_GEX.obs.index)}')
print(f'Total number of GEX UMIs: {total_GEX_UMIs}')

In [ ]:
### SB DATA OF ALL CELL BARCODES ###

# Load the SB counts h5 file and initialize SB Pandas
gel_2_deep_SB_counts_path = 'gel_2_deep/gel_2_deep_all_counts_MS.h5'
gel_2_deep_SB_counts_h5 = h5py.File(gel_2_deep_SB_counts_path, 'r')
num_matched_CB_SB_pairs = gel_2_deep_SB_counts_h5['matrix']['umi'].shape[0]
total_SB_UMIs = num_matched_CB_SB_pairs

# Try to remap CB, return 'ERROR' if CB is not direct match
def remap_CB(CB):
    try:
        return reverse_matching_3p_10X_CBs[CB]
    except:
        return 'ERROR'

spatial_CB_list = list(map(remap_CB, map(lambda CB: CB.decode('UTF-8'), gel_2_deep_SB_counts_h5['lists/cb_list'][:])))

# Get data from SB count matrix
cb_indices, reads, sb_indices, nUMIs =  gel_2_deep_SB_counts_h5['matrix'].values()

# Initialize cumulative SB umi dictionary
spatial_CB_nUMIs = defaultdict(lambda: 0)

# Iterate over all (cb, sb, umi) pairs and count number of UMIs
for i in range(num_matched_CB_SB_pairs):
    # Need to re-index because R indexing starts at 1
    CB = spatial_CB_list[cb_indices[i] - 1]

    # Add UMIs to CB
    spatial_CB_nUMIs[CB] += 1

# Count total number of UMIs and number of non-CB-matched UMIs
missed_SB_UMIs = spatial_CB_nUMIs['ERROR']

# Print SB UMI metrics
print(f'Number of observed CBs in the SB library: {len(spatial_CB_list)}')
print(f'Number of CBs with SB UMIs: {len(spatial_CB_nUMIs.keys()) - 1}')
print(f'Number of observed CBs that are not valid: {op.countOf(spatial_CB_list, "ERROR")}')
print(f'Number of SB UMIs that cannot be exactly mapped to a cb: {missed_SB_UMIs}, {round(missed_SB_UMIs / num_matched_CB_SB_pairs, 3)}')

# Convert dictionary to df
spatial_CB_df = pd.DataFrame.from_dict(data = spatial_CB_nUMIs, orient = 'index').rename(columns={0: 'Total SB UMIs'})

In [ ]:
### COMBINED DATA OF ALL CBS ###

combined_UMI_df = pd.read_csv('gel_2_deep_GEX_SB_UMI_counts.csv', index_col='CB').sort_values('Total UMIs', ascending = False)
combined_UMI_df

# # Merge GEX and SB UMI data
# combined_UMI_df = spatial_CB_df.merge(gel_2_GEX.obs, how = 'outer', left_index = True, right_index = True).fillna(0)
# combined_UMI_df['Total UMIs'] = combined_UMI_df['Total GEX UMIs'] + combined_UMI_df['Total SB UMIs']
# combined_UMI_df.index = combined_UMI_df.index.rename('CB')
# combined_UMI_df = combined_UMI_df.sort_values('Total UMIs', ascending = False).drop('ERROR')
# combined_UMI_df.to_csv('gel_2_deep_GEX_SB_UMI_counts.csv')

In [ ]:
# # Get CellRanger called CBs
CellRanger_called_CBs_df = pd.read_csv('gel_2/CellRanger_gel_2/gel_2_CellRanger_filtered_feature_bc_matrix/barcodes.tsv.gz', header = None).rename(columns={0: 'CB'})
CellRanger_called_CBs = set([CB[:-2] for CB in CellRanger_called_CBs_df['CB']])

# combined_UMI_df['Droplet'] = combined_UMI_df.index.isin(CellRanger_called_CBs)
# combined_UMI_df.to_csv('gel_2_deep_GEX_SB_UMI_counts.csv')

In [ ]:
fig, (GEX_ax, SB_ax, total_ax, comb_ax) = plt.subplots(nrows = 4, ncols = 1, figsize = (5,16), sharex=True, sharey = True)

# Plot CB Rank plot by GEX UMIs
combined_UMI_df.sort_values('Total GEX UMIs', ascending = False)['Total GEX UMIs'].plot(ax = GEX_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'GEX UMIs', color = 'green')

GEX_ax.set_title('CB Rank Plot by Total GEX UMIs from CellRanger')
GEX_ax.set_ylabel('Total GEX UMIs')
GEX_ax.set_xlabel('CB Rank')
GEX_ax.legend(loc=0)
GEX_ax.axvline(x = len(CellRanger_called_CBs), color = 'red')

# Plot CB rank plot of SB UMIs, still ranked by GEX UMIs
combined_UMI_df.sort_values('Total GEX UMIs', ascending = False)['Total SB UMIs'].plot(ax = SB_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'SB UMIs', color = 'orange')

SB_ax.set_title('CB Rank Plot by Total SB UMIs')
SB_ax.set_ylabel('Total SB UMIs')
SB_ax.set_xlabel('CB Rank')
SB_ax.legend(loc=0)
SB_ax.axvline(x = len(CellRanger_called_CBs), color = 'red')

# Plot CB rank plot of Total UMIs, still ranked by GEX UMIs
combined_UMI_df.sort_values('Total GEX UMIs', ascending = False)['Total UMIs'].plot(ax = total_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'Total UMIs', color = 'blue')

total_ax.set_title('CB Rank Plot by Total UMIs')
total_ax.set_ylabel('Total UMIs')
total_ax.set_xlabel('CB Rank')
total_ax.legend(loc=0)
total_ax.axvline(x = len(CellRanger_called_CBs), color = 'red')

# Plot CB rank plot by respective metrics
combined_UMI_df.sort_values('Total GEX UMIs', ascending = False)['Total GEX UMIs'].plot(ax = comb_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'GEX UMIs', color = 'green')
combined_UMI_df.sort_values('Total SB UMIs', ascending = False)['Total SB UMIs'].plot(ax = comb_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'SB UMIs', color = 'orange')
combined_UMI_df['Total UMIs'].plot(ax = comb_ax, kind = 'line', logx=True, logy=True, use_index = False, label = 'Total UMIs', color = 'blue')

comb_ax.set_title('CB Rank Plot by Total UMIs')
comb_ax.set_ylabel('Total UMIs')
comb_ax.set_xlabel('CB Rank')
comb_ax.legend(loc=0)

fig.savefig('gel_2_deep/UMI_distribution.png')

In [ ]:
def in_circle(x,y):
    x_center, y_center = 4000, 3000
    radius = 1600
    return np.sqrt((x - x_center)**2 + (y - y_center)**2) < radius

cutoff_mapped_nuc_df = pd.read_csv('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/Slidetags/gel_2_deep/CellRanger_gel_2_deep_cutoff/gel_2_deep_cutoff_info.csv', index_col = 'CB')
cluster_18_CBs = set(cutoff_mapped_nuc_df[cutoff_mapped_nuc_df['seurat_clusters'] == 18].index)
cutoff_mapped_nuc_df['in_circle'] = cutoff_mapped_nuc_df.apply(lambda x: in_circle(x['x_um'], x['y_um']), axis = 1)
c18_df = cutoff_mapped_nuc_df[cutoff_mapped_nuc_df['seurat_clusters'] == 18].reset_index()
c18_in_circle = c18_df[c18_df['in_circle'] == True]
cutoff_in_circle = cutoff_mapped_nuc_df[cutoff_mapped_nuc_df['in_circle'] == True]
c18_in_circle


In [ ]:
nrows = 1
ncols = 1
fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize = (6 * ncols, 6 * nrows))

ax.set_title(f'Cluster 18 - Good Positions')
c18_in_circle.plot.scatter(x = 'x_um', y = 'y_um', ax = ax, s = 10, alpha = 1, color = 'Blue')
ax.set_xlim(0, 6500)
ax.set_ylim(0,6500)

circle = plt.Circle((3250, 3600), 2600, color='r', fill = False, label = 'Puck')
ax.add_patch(circle)

circle = plt.Circle((4000, 3000), 1600, color='g', fill = False, label = 'Possible nuclei locations')
ax.add_patch(circle)

# circle = plt.Circle((4000, 3000), 1000, color='b', fill = False)
# ax.add_patch(circle)

lc = LineCollection([[(4000 - 1000, 3000), (4000 + 1000, 3000)]], colors = ['b'], linewidths = 1, label = 'Simulated Line Locations')
ax.add_collection(lc)



plt.legend()
plt.savefig('../BarcodeBender/official_data/gel_2_c18_positions.png')

plt.show()

In [ ]:
cutoff_mapped_nuc_df[cutoff_mapped_nuc_df.index.isin(cutoff_in_circle.index)]

In [ ]:
### DEFINE DROPLETS IN DIFFERENT REGIMES ###

# Ambient regime starts after CellRanger-called droplets
min_ambient = 40000 #len(CellRanger_called_CBs)

# EStimated 100,000 ambient droplets from graph
max_ambient = 80000

# Isolate ambient CBs and write them to disk
ambient_CBs = list(combined_UMI_df.sort_values('Total GEX UMIs', ascending = False).iloc[min_ambient: max_ambient].index)
# with open('gel_2/ambient_noise_CBs.txt', 'w') as f:
#     for CB in ambient_CBs:
#         f.write(f'{CB}\n')

# CBs mapped via gel_2_cutoff
cutoff_mapped_nuc_df = pd.read_csv('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/Slidetags/gel_2_deep/CellRanger_gel_2_deep_cutoff/gel_2_deep_cutoff_info.csv', index_col = 'CB')
confidently_mapped_CBs = list(cutoff_in_circle.index)
cluster_18_CBs = list(cutoff_mapped_nuc_df[cutoff_mapped_nuc_df['seurat_clusters'] == 18].index)

print(f'{len(confidently_mapped_CBs)} confidently-mapped CBs')
print(f'{len(cluster_18_CBs)} cluster 18 CBs')

# with open('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/BarcodeBender/SpatialInference/cluster_18_CBs.csv', 'w', newline='') as f:
#     write = csv.writer(f)
#     write.writerow(cluster_18_CBs)

In [ ]:
cutoff_mapped_nuc_df[cutoff_mapped_nuc_df.index.isin(cutoff_in_circle.index)].to_csv('../BarcodeBender/gel_2_deep_mappable_data/gel_2_deep_all_info.csv')

In [ ]:
with open('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/BarcodeBender/SpatialInference/cluster_18_CBs.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(cluster_18_CBs)
pd.read_csv('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/BarcodeBender/SpatialInference/cluster_18_CBs.csv').T.to_csv('/Users/jacobshapiro/Library/Mobile Documents/com~apple~CloudDocs/Documents/Harvard/Chen Lab/BarcodeBender/SpatialInference/cluster_18_CBs.csv')

In [ ]:
# ### GET SPECIFIC SB UMI COUNTS IN CBS ###

# # Initialize cumulative SB umi dictionary
# spatial_SBs_in_CBs = defaultdict(lambda: defaultdict(lambda: 0))
# spatial_SB_list = list(map(lambda SB: SB.decode('UTF-8'), gel_2_deep_SB_counts_h5['lists/sb_list'][:]))

# # Iterate over all (cb, sb, umi) pairs and count number of UMIs
# for i in range(num_matched_CB_SB_pairs):
#     # Need to re-index because R indexing starts at 1
#     CB = spatial_CB_list[cb_indices[i] - 1]
#     SB = spatial_SB_list[sb_indices[i] - 1]

#     # Add UMIs to CB
#     spatial_SBs_in_CBs[CB][SB] += 1

# # Convert to AnnData
# indptr = [0]
# indices = []
# data = []
# SB_indices = {}

# for CB, SBs in spatial_SBs_in_CBs.items():
#     for SB, UMIs in SBs.items():
#         SB_index = SB_indices.setdefault(SB, len(SB_indices))
#         indices.append(SB_index)
#         data.append(float(UMIs))
#     indptr.append(len(indices))

# CB_SB_counts_AD = anndata.AnnData(sparse.csr_matrix((data, indices, indptr)))


# obs_CBs = list(spatial_SBs_in_CBs.keys())
# vars_SBs = list(SB_indices.keys())
# CB_SB_counts_AD.obs_names = obs_CBs
# CB_SB_counts_AD.var_names = vars_SBs
# CB_SB_counts_AD.write('gel_2_deep/CB_SB_counts.h5ad')

In [ ]:
CB_SB_counts_AD = anndata.read_h5ad('../BarcodeBender/official_data/gel_2_deep_all_CB_SB_counts.h5ad')
CB_SB_counts_AD.write('../BarcodeBender/gel_2_deep_mappable_data/gel_2_deep_all_CB_SB_counts.h5ad')

ambient_CB_SB_counts_AD = CB_SB_counts_AD[ambient_CBs, :]
ambient_CB_SB_counts_AD.write('../BarcodeBender/gel_2_deep_mappable_data/gel_2_deep_ambient_CB_SB_counts.h5ad')
print(ambient_CB_SB_counts_AD)

confidently_mapped_CB_SB_counts_AD = CB_SB_counts_AD[confidently_mapped_CBs, :]
confidently_mapped_CB_SB_counts_AD.write('../BarcodeBender/gel_2_deep_mappable_data/gel_2_deep_confidently_mapped_CB_SB_counts.h5ad')
print(confidently_mapped_CB_SB_counts_AD)

# cluster_18_CB_SB_counts_AD = CB_SB_counts_AD[cluster_18_CBs, :]
# cluster_18_CB_SB_counts_AD.write('../BarcodeBender/official_data/gel_2_deep_cluster_18_CB_SB_counts.h5ad')
# print(cluster_18_CB_SB_counts_AD)

In [ ]:

confidently_mapped_CB_SB_counts_top_SBs_AD = confidently_mapped_CB_SB_counts_AD[confidently_mapped_CBs, top_SBs]
confidently_mapped_CB_SB_counts_top_SBs_AD.write('../BarcodeBender/gel_2_deep_mappable_data/gel_2_deep_confidently_mapped_CB_SB_counts_top_SBs.h5ad')
print(confidently_mapped_CB_SB_counts_top_SBs_AD)

# cluster_18_CB_SB_counts_AD = CB_SB_counts_AD[cluster_18_CBs, :]
# cluster_18_CB_SB_counts_AD.write('gel_2_deep/cluster_18_CB_SB_counts.h5ad')
# print(cluster_18_CB_SB_counts_AD)

# cluster_18_CB_SB_counts_top_SBs_AD = cluster_18_CB_SB_counts_AD[:,cluster_18_CB_SB_counts_AD.X.sum(axis = 0) > 5]
# cluster_18_top_SBs = list(cluster_18_CB_SB_counts_top_SBs_AD.var_names)
# cluster_18_CB_SB_counts_top_SBs_AD.write('gel_2_deep/cluster_18_CB_SB_counts_top_SBs.h5ad')
# print(cluster_18_CB_SB_counts_top_SBs_AD)

# ambient_CB_SB_counts_top_SBs_AD = CB_SB_counts_AD[ambient_CBs, top_SBs]
# ambient_CB_SB_counts_top_SBs_AD.write('gel_2_deep/ambient_CB_SB_counts_top_SBs.h5ad')
# print(ambient_CB_SB_counts_top_SBs_AD)

In [ ]:
# Calculate Top SBs
top_SBs = ambient_CB_SB_counts_AD.var_names[ambient_CB_SB_counts_AD.X.sum(axis = 0).A[0] > 10]
print(f'Number of Top SBs: {len(top_SBs)}')
# ambient_CB_SB_counts_top_SBs_AD = ambient_CB_SB_counts_AD[:, top_SBs]
# ambient_CB_SB_counts_top_SBs_AD.write('../BarcodeBender/official_data/gel_2_deep_ambient_CB_SB_counts_top_SBs.h5ad')

# cluster_18_CB_SB_counts_top_SBs_AD = cluster_18_CB_SB_counts_AD[:, top_SBs]
# cluster_18_CB_SB_counts_top_SBs_AD.write('../BarcodeBender/official_data/gel_2_deep_cluster_18_CB_SB_counts_top_SBs.h5ad')
# print(CB_SB_counts_AD)
# print(confidently_mapped_CB_SB_counts_AD)
# print(cluster_18_CB_SB_counts_AD)
# print(cluster_18_CB_SB_counts_top_SBs_AD)
# print(ambient_CB_SB_counts_AD)
# print(ambient_CB_SB_counts_top_SBs_AD)

In [ ]:
chi_ambient = np.array(ambient_CB_SB_counts_AD.X.sum(axis = 0)).squeeze()
chi_ambient = chi_ambient[chi_ambient > 0]
log_chi_ambient = np.log(chi_ambient)

x = np.arange(
    0,
    np.ceil(log_chi_ambient.max()) + 0.01,
    0.1
)

plt.hist(log_chi_ambient, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in all ambient droplets')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_chi_ambient_UMI_hist.png')
plt.show()

In [ ]:
chi_ambient_top_SBs = np.array(ambient_CB_SB_counts_top_SBs_AD.X.sum(axis = 0)).squeeze()

log_chi_ambient_top_SBs = np.log(chi_ambient_top_SBs)

# with open('../BarcodeBender/chi_ambient_rho_SBs.txt', 'w') as f:
#     for rho_SB in scaled_chi_ambient_top_SBs:
#         f.write(f'{rho_SB}\n')

x = np.arange(
    0,
    np.ceil(log_chi_ambient_top_SBs.max()) + 0.01,
    0.1
)

chi_ambient_top_SBs_loc = np.mean(log_chi_ambient_top_SBs)
chi_ambient_top_SBs_scale = np.std(log_chi_ambient_top_SBs)
print(np.mean(log_chi_ambient_top_SBs))

plt.hist(log_chi_ambient_top_SBs, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in all ambient droplets, only top SBs;\nMean: {round(chi_ambient_top_SBs_loc, 5)}, Std. Dev.: {round(chi_ambient_top_SBs_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_chi_ambient_top_SBs_UMI_hist.png')
plt.show()

In [ ]:
scaled_chi_ambient_top_SBs.min()

In [ ]:
chi_ambient_top_SBs = np.array(ambient_CB_SB_counts_top_SBs_AD.X.sum(axis = 0)).squeeze()
scaled_chi_ambient_top_SBs = chi_ambient_top_SBs * 203 / 40000
scaled_chi_ambient_top_SBs = scaled_chi_ambient_top_SBs[scaled_chi_ambient_top_SBs > 0]
log_scaled_chi_ambient_top_SBs = np.log(scaled_chi_ambient_top_SBs)

with open('../BarcodeBender/official_data/gel_2_deep_scaled_chi_ambient_rho_top_SBs.txt', 'w') as f:
    for rho_SB in scaled_chi_ambient_top_SBs:
        f.write(f'{rho_SB}\n')

x = np.arange(
    np.floor(log_scaled_chi_ambient_top_SBs.min()) - 0.01,
    np.ceil(log_scaled_chi_ambient_top_SBs.max()) + 0.01,
    0.1
)

scaled_chi_ambient_top_SBs_loc = np.mean(log_scaled_chi_ambient_top_SBs)
scaled_chi_ambient_top_SBs_scale = np.std(log_scaled_chi_ambient_top_SBs)

plt.hist(log_scaled_chi_ambient_top_SBs, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Scaled Distribution of Log(UMIs) of SBs present in all ambient droplets, only top SBs;\nMean: {round(scaled_chi_ambient_top_SBs_loc, 5)}, Std. Dev.: {round(scaled_chi_ambient_top_SBs_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_scaled_chi_ambient_top_SBs_UMI_hist.png')
plt.show()

In [ ]:
ambient_umi_counts = np.array(ambient_CB_SB_counts_AD.X.sum(axis=1)).squeeze()
ambient_umi_counts = ambient_umi_counts[ambient_umi_counts > 0]
log_ambient_umi_counts = np.log(ambient_umi_counts)

x = np.arange(
    0,
    np.ceil(log_ambient_umi_counts.max()) + 0.01,
    0.01
)

ambient_CB_UMI_loc = np.mean(log_ambient_umi_counts)
ambient_CB_UMI_scale = np.std(log_ambient_umi_counts)

plt.hist(log_ambient_umi_counts, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Ambient Droplets;\nMean: {round(ambient_CB_UMI_loc, 5)}, Std. Dev.: {round(ambient_CB_UMI_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_ambient_droplets_log_SB_UMI_hist.png')
plt.show()

In [ ]:
ambient_umi_counts_top_SBs = np.array(ambient_CB_SB_counts_top_SBs_AD.X.sum(axis=1)).squeeze()
ambient_umi_counts_top_SBs = ambient_umi_counts_top_SBs[ambient_umi_counts_top_SBs > 0]
log_ambient_umi_counts_top_SBs = np.log(ambient_umi_counts_top_SBs)


x = np.arange(
    0,
    np.ceil(log_ambient_umi_counts_top_SBs.max()) + 0.01,
    0.01
)

print(np.mean(ambient_umi_counts_top_SBs))
ambient_CB_top_SBs_UMI_loc = np.mean(log_ambient_umi_counts_top_SBs)
ambient_CB_top_SBs_UMI_scale = np.std(log_ambient_umi_counts_top_SBs)

plt.hist(log_ambient_umi_counts_top_SBs, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Ambient Droplets with Top SBs;\nMean: {round(ambient_CB_top_SBs_UMI_loc, 5)}, Std. Dev.: {round(ambient_CB_top_SBs_UMI_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_ambient_droplets_top_SBs_log_SB_UMI_hist.png')
plt.show()

In [ ]:
confidently_mapped_umi_counts = np.array(confidently_mapped_CB_SB_counts_AD.X.sum(axis = 1)).squeeze()
log_confidently_mapped_umi_counts = np.log(confidently_mapped_umi_counts)
log_confidently_mapped_umi_counts = log_confidently_mapped_umi_counts[log_confidently_mapped_umi_counts > 0]
x = np.arange(
    0,
    np.ceil(log_confidently_mapped_umi_counts.max()) + 0.01,
    0.1
)
print(np.mean(log_confidently_mapped_umi_counts))
k = gaussian_kde(log_confidently_mapped_umi_counts)
density = k.evaluate(x)
log_peak_ind = np.argmax(density)
log_peak = x[log_peak_ind]

confidently_mapped_UMI_loc = log_peak
confidently_mapped_UMI_scale = np.std(log_confidently_mapped_umi_counts)

plt.hist(log_confidently_mapped_umi_counts, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Confidently Mapped Droplets;\nMean: {round(confidently_mapped_UMI_loc, 2)}, Std. Dev.: {round(confidently_mapped_UMI_scale, 2)}')
plt.savefig('gel_2_deep/confidently_mapped_log_SB_UMI_hist.png')
plt.show()

In [ ]:
confidently_mapped_top_SBs_umi_counts = np.array(confidently_mapped_CB_SB_counts_top_SBs_AD.X.sum(axis = 1)).squeeze()
log_confidently_mapped_top_SBs_umi_counts = np.log(confidently_mapped_top_SBs_umi_counts)
log_confidently_mapped_top_SBs_umi_counts = log_confidently_mapped_top_SBs_umi_counts[log_confidently_mapped_top_SBs_umi_counts > 0]
x = np.arange(
    0,
    np.ceil(log_confidently_mapped_top_SBs_umi_counts.max()) + 0.01,
    0.1
)
print(np.mean(log_confidently_mapped_top_SBs_umi_counts))
k = gaussian_kde(log_confidently_mapped_top_SBs_umi_counts)
density = k.evaluate(x)
log_peak_ind = np.argmax(density)
log_peak = x[log_peak_ind]

confidently_mapped_top_SBs_UMI_loc = log_peak
confidently_mapped_top_SBs_UMI_scale = np.std(log_confidently_mapped_top_SBs_umi_counts)

plt.hist(log_confidently_mapped_top_SBs_umi_counts, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Confidently Mapped Droplets with Top SBs;\nMean: {round(confidently_mapped_top_SBs_UMI_loc, 4)}, Std. Dev.: {round(confidently_mapped_top_SBs_UMI_scale, 4)}')
# plt.savefig('gel_2_deep/confidently_mapped_top_SBs_log_SB_UMI_hist.png')
plt.show()

In [ ]:
cluster_18_umi_counts = np.array(cluster_18_CB_SB_counts_AD.X.sum(axis = 1)).squeeze()
cluster_18_umi_counts = cluster_18_umi_counts[cluster_18_umi_counts > 0]
log_cluster_18_umi_counts = np.log(cluster_18_umi_counts)
x = np.arange(
    0,
    np.ceil(log_cluster_18_umi_counts.max()) + 0.01,
    0.1
)


cluster_18_UMI_loc = np.mean(log_cluster_18_umi_counts)
cluster_18_UMI_scale = np.std(log_cluster_18_umi_counts)

plt.hist(log_cluster_18_umi_counts, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Cluster 18 Droplets;\nMean: {round(cluster_18_UMI_loc, 5)}, Std. Dev.: {round(cluster_18_UMI_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_cluster_18_log_SB_UMI_hist.png')
plt.show()

In [ ]:
cluster_18_top_SBs_umi_counts = np.array(cluster_18_CB_SB_counts_top_SBs_AD.X.sum(axis = 1)).squeeze()
cluster_18_top_SBs_umi_counts = cluster_18_top_SBs_umi_counts[cluster_18_top_SBs_umi_counts > 0]
log_cluster_18_top_SBs_umi_counts = np.log(cluster_18_top_SBs_umi_counts)

x = np.arange(
    0,
    np.ceil(log_cluster_18_top_SBs_umi_counts.max()) + 0.01,
    0.1
)

print(np.mean(cluster_18_top_SBs_umi_counts))
cluster_18_top_SBs_UMI_loc = np.mean(log_cluster_18_top_SBs_umi_counts)
cluster_18_top_SBs_UMI_scale = np.std(log_cluster_18_top_SBs_umi_counts)

plt.hist(log_cluster_18_top_SBs_umi_counts, bins = x)
plt.ylabel('Number of Droplets')
plt.xlabel('Log(SB UMIs)')
plt.title(f'Distribution of Log(SB UMIs) of Cluster 18 Droplets with Top SBs;\nMean: {round(cluster_18_top_SBs_UMI_loc, 5)}, Std. Dev.: {round(cluster_18_top_SBs_UMI_scale, 5)}')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_cluster_18_top_SBs_log_SB_UMI_hist.png')
plt.show()

In [ ]:
chi_confidently_mapped = np.array(confidently_mapped_CB_SB_counts_AD.X.sum(axis = 0)).squeeze()
chi_confidently_mapped_normalized = (chi_confidently_mapped / chi_confidently_mapped.sum())

log_chi_confidently_mapped = np.log(chi_confidently_mapped)
log_chi_confidently_mapped = log_chi_confidently_mapped[log_chi_confidently_mapped > 0]
x = np.arange(
    0, np.ceil(log_chi_confidently_mapped.max()) + 0.01, 0.2)#     np.ceil(log_chi_cluster_18.max()) + 0.01,
#     1
# )

k = gaussian_kde(log_chi_confidently_mapped)
density = k.evaluate(x)
log_peak_ind = np.argmax(density)
log_peak = x[log_peak_ind]

chi_cluster_18_loc = np.exp(log_peak)
chi_cluster_18_scale = np.std(log_chi_confidently_mapped)

plt.hist(log_chi_confidently_mapped, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in Confidently Mapped Droplets')
plt.savefig('gel_2_deep/chi_confidently_mapped_UMI_hist.png')
plt.show()

In [ ]:
chi_confidently_mapped_top_SBs = np.array(confidently_mapped_CB_SB_counts_top_SBs_AD.X.sum(axis = 0)).squeeze()
chi_confidently_mapped_top_SBs_normalized = (chi_confidently_mapped_top_SBs / chi_confidently_mapped_top_SBs.sum())

log_chi_confidently_mapped_top_SBs = np.log(chi_confidently_mapped_top_SBs)
log_chi_confidently_mapped_top_SBs = log_chi_confidently_mapped_top_SBs[log_chi_confidently_mapped_top_SBs > 0]
x = np.arange(
    0, np.ceil(log_chi_confidently_mapped_top_SBs.max()) + 0.01, 0.1)#     np.ceil(log_chi_cluster_18.max()) + 0.01,
#     1
# )

k = gaussian_kde(log_chi_confidently_mapped_top_SBs)
density = k.evaluate(x)
log_peak_ind = np.argmax(density)
log_peak = x[log_peak_ind]

chi_cluster_18_loc = np.exp(log_peak)
chi_cluster_18_scale = np.std(log_chi_confidently_mapped_top_SBs)

plt.hist(log_chi_confidently_mapped_top_SBs, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in Confidently Mapped Droplets with Top SBs')
plt.savefig('gel_2_deep/chi_confidently_mapped_top_SBs_UMI_hist.png')
plt.show()

In [ ]:
chi_cluster_18 = np.array(cluster_18_CB_SB_counts_AD.X.sum(axis = 0)).squeeze()
chi_cluster_18_normalized = (chi_cluster_18 / chi_cluster_18.sum())

log_chi_cluster_18 = np.log(chi_cluster_18)
log_chi_cluster_18 = log_chi_cluster_18[log_chi_cluster_18 > 0]
x = np.arange(
    0, np.ceil(log_chi_cluster_18.max()) + 0.01, 0.1)#     np.ceil(log_chi_cluster_18.max()) + 0.01,
#     1
# )

k = gaussian_kde(log_chi_cluster_18)
density = k.evaluate(x)
log_peak_ind = np.argmax(density)
log_peak = x[log_peak_ind]

chi_cluster_18_loc = np.exp(log_peak)
chi_cluster_18_scale = np.std(log_chi_cluster_18)

plt.hist(log_chi_cluster_18, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in Cluster 18 Droplets')
plt.savefig('gel_2_deep/chi_cluster_18_UMI_hist.png')
plt.show()

In [ ]:
chi_cluster_18_top_SBs = np.array(cluster_18_CB_SB_counts_top_SBs_AD.X.sum(axis = 0)).squeeze()
chi_cluster_18_top_SBs = chi_cluster_18_top_SBs[chi_cluster_18_top_SBs > 0]
log_chi_cluster_18_top_SBs = np.log(chi_cluster_18_top_SBs)
x = np.arange(
    0, np.ceil(log_chi_cluster_18_top_SBs.max()) + 0.01, 0.1)

chi_cluster_18_top_SBs_loc = np.mean(log_chi_cluster_18_top_SBs)
chi_cluster_18_top_SBs_scale = np.std(log_chi_cluster_18_top_SBs)

plt.hist(log_chi_cluster_18_top_SBs, bins = x)
plt.ylabel('Number of SBs')
plt.xlabel('Log(UMIs)')
plt.title(f'Distribution of Log(UMIs) of SBs present in Cluster 18 Droplets, top SBs')
plt.savefig('../BarcodeBender/official_data/gel_2_deep_chi_cluster_18_top_SBs_UMI_hist.png')
plt.show()

In [ ]:
### GET COORDINATES OF SPATIAL BARCODES ###

# Get SB list from h5
sb_list = list(map(lambda CB: CB.decode('UTF-8'), gel_2_deep_SB_counts_h5['lists/sb_list'][:]))

# Extract and convert coordinates from h5
scaling_factor = 2 * 0.73
x_list = list(map(lambda x_coord: scaling_factor * x_coord, gel_2_deep_SB_counts_h5['lists/x_list'][:]))
y_list = list(map(lambda y_coord: scaling_factor * y_coord, gel_2_deep_SB_counts_h5['lists/y_list'][:]))

# Compile into dataframe
SB_coordinates = pd.DataFrame(index=sb_list, data={'x_um': x_list, 'y_um': y_list})

### WRITE SB COORDINATES TO DISK ###

# with open('../BarcodeBender/ordered_SB_coordinates.txt', 'w') as f:
#     for SB in confidently_mapped_CB_SB_counts_AD.var_names:
#         x_coord, y_coord = SB_coordinates.loc[SB]
#         f.write(f'{SB}, {x_coord}, {y_coord}\n')

with open('../BarcodeBender/official_data/ordered_top_SB_coordinates.txt', 'w') as f:
    for SB in cluster_18_CB_SB_counts_top_SBs_AD.var_names:
        x_coord, y_coord = SB_coordinates.loc[SB]
        f.write(f'{SB}, {x_coord}, {y_coord}\n')

In [ ]:
### AMBIENT BACKGROUND VISUALIZATION USING ONLY TOP SBs ###

# Add UMIs for all SBs in the ambient droplets
ambient_background = defaultdict(lambda: 0)

for CB in ambient_CBs:
    SBs = spatial_SBs_in_CBs[CB]
    for SB, UMIs in SBs.items():
        if SB in top_SBs:
            ambient_background[SB] += UMIs

print(f'Plotting average background...\n')

# Plot UMIs in XY coordinates
SBs, x_coords, y_coords, counts = [], [], [], []
for SB, UMIs in ambient_background.items():
    x_coord, y_coord = SB_coordinates.loc[SB]
    SBs.append(SB), x_coords.append(x_coord), y_coords.append(y_coord), counts.append(UMIs)

print(f'Number of SBs: {len(x_coords)}')

# Plot locations of SBs
fig, ax = plt.subplots(figsize = (6,6))
ax.set_title(f'Gel 2 Deep - Top SB Locations')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
ax.scatter(x_coords, y_coords, s = 2, alpha = 1, cmap = 'blues', edgecolors = 'black', linewidths = 0.1)
circle = plt.Circle((3200, 3250), 2500, color='r', fill = False)
ax.add_patch(circle)
fig.savefig('../BarcodeBender/official_data/gel_2_deep_top_SB_locations.png')

# Convert to DF for sorting -important to get highest UMIs last
background_plotting_df = pd.DataFrame(index = SBs, data = {'x_coords': x_coords, 'y_coords': y_coords, 'counts': counts}).sort_values('counts', ascending=True)

# Plot background UMIs
fig, ax = plt.subplots()
ax.set_title(f'Ambient Background: UMIs of Top SBs in Ambient Regime')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(background_plotting_df['x_coords'], background_plotting_df['y_coords'], s = 10, c = background_plotting_df['counts'], alpha = 0.5, cmap = 'Blues')
plt.colorbar(sc)

# Save to disk
fig.savefig('../BarcodeBender/official_data/gel_2_deep_ambient_background_top_SBs_visualization.png')

c = np.array(background_plotting_df['counts'])
total_matched_SB_UMIs = sum(c)
normalized_c = c / total_matched_SB_UMIs
background_plotting_df['counts_normalized_background'] = normalized_c

# Plot background UMIs normalied - linear
fig, ax = plt.subplots()
ax.set_title(f'Ambient Background Top SBs Linear Normalized')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc_linear = ax.scatter(background_plotting_df['x_coords'], background_plotting_df['y_coords'], s = 10, c = background_plotting_df['counts_normalized_background'], alpha = 0.5, cmap = 'Blues')
fig.colorbar(sc_linear)


# Save to disk
fig.savefig('../BarcodeBender/official_data/gel_2_deep_ambient_background_top_SBs_normalized_linear_visualization.png')

In [ ]:
### AMBIENT BACKGROUD VISUALIZATION ###

# Add UMIs for all SBs in the ambient droplets
ambient_background = defaultdict(lambda: 0)

for CB in ambient_CBs:
    SBs = spatial_SBs_in_CBs[CB]
    for SB, UMIs in SBs.items():
        ambient_background[SB] += UMIs

print(f'Plotting average background...\n')

# Plot UMIs in XY coordinates
SBs, x_coords, y_coords, counts = [], [], [], []

for SB, UMIs in ambient_background.items():
    x_coord, y_coord = SB_coordinates.loc[SB]
    SBs.append(SB), x_coords.append(x_coord), y_coords.append(y_coord), counts.append(UMIs)

# Convert to DF for sorting -important to get highest UMIs last
background_plotting_df = pd.DataFrame(index = SBs, data = {'x_coords': x_coords, 'y_coords': y_coords, 'counts': counts}).sort_values('counts', ascending=True)

# Plot background UMIs
fig, ax = plt.subplots()
ax.set_title(f'Ambient Background: UMIs of SBs in Ambient Regime')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(background_plotting_df['x_coords'], background_plotting_df['y_coords'], s = 10, c = background_plotting_df['counts'], alpha = 0.5, cmap = 'Blues')
plt.colorbar(sc)

# Save to disk
fig.savefig('gel_2_deep/ambient_background_visualization.png')

c = np.array(background_plotting_df['counts'])
total_matched_SB_UMIs = sum(c)
normalized_c = c / total_matched_SB_UMIs
background_plotting_df['counts_normalized_background'] = normalized_c

# Plot background UMIs normalied - linear
fig, ax = plt.subplots()
ax.set_title(f'Ambient Background Linear Normalized')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc_linear = ax.scatter(background_plotting_df['x_coords'], background_plotting_df['y_coords'], s = 10, c = background_plotting_df['counts_normalized_background'], alpha = 0.5, cmap = 'Blues')
fig.colorbar(sc_linear)

# Save to disk
fig.savefig('gel_2_deep/ambient_background_normalized_linear_visualization.png')

In [ ]:
### CONFIDENTLY MAPPABLED SB VISUALIZATION ###

# Add UMIs for all SBs in the confident droplets
mappable_UMIs = defaultdict(lambda: 0)

for CB in confidently_mapped_CBs:
    SBs = spatial_SBs_in_CBs[CB]
    for SB, UMIs in SBs.items():
        mappable_UMIs[SB] += UMIs

print(f'Plotting mappable UMI distribution...\n')

# Plot UMIs in XY coordinates
SBs, x_coords, y_coords, counts = [], [], [], []

for SB, UMIs in mappable_UMIs.items():
    x_coord, y_coord = SB_coordinates.loc[SB]
    SBs.append(SB), x_coords.append(x_coord), y_coords.append(y_coord), counts.append(UMIs)

# Convert to DF for sorting -important to get highest UMIs last
confident_plotting_df = pd.DataFrame(index = SBs, data = {'x_coords': x_coords, 'y_coords': y_coords, 'counts': counts}).sort_values('counts', ascending=True)

# Plot mappable UMIs
fig, ax = plt.subplots()
ax.set_title(f'Confident UMIs: UMIs of Droplets that can be Confidently Mapped')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(confident_plotting_df['x_coords'], confident_plotting_df['y_coords'], s = 10, c = confident_plotting_df['counts'], alpha = 0.5, cmap = 'Blues')
plt.colorbar(sc)

# Save to disk
fig.savefig('gel_2_deep/mappable_UMI_visualization.png')

c = np.array(confident_plotting_df['counts'])
total_matched_SB_UMIs = sum(c)
normalized_c = c / total_matched_SB_UMIs
confident_plotting_df['counts_normalized_mappable'] = normalized_c

# Plot mappable UMIs normalied - linear
fig, ax = plt.subplots()
ax.set_title(f'Mappable UMIs Linear Normalized')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc_linear = ax.scatter(confident_plotting_df['x_coords'], confident_plotting_df['y_coords'], s = 10, c = confident_plotting_df['counts_normalized_mappable'], alpha = 0.5, cmap = 'Blues')
fig.colorbar(sc_linear)

# Save to disk
fig.savefig('gel_2_deep/mappable_UMI_visualization_linear_visualization.png')

In [ ]:
### WHOLE EXPERIMENT VISUALIZATION ###

# Add UMIs for all SBs
experiment_SBs = defaultdict(lambda: 0)

for SBs in spatial_SBs_in_CBs.values():
    for SB, UMIs in SBs.items():
        experiment_SBs[SB] += UMIs

print(f'Plotting whole experiment UMIs...\n')

# Plot UMIs in XY coordinates
SBs, x_coords, y_coords, counts = [], [], [], []

for SB, UMIs in experiment_SBs.items():
    x_coord, y_coord = SB_coordinates.loc[SB]
    SBs.append(SB), x_coords.append(x_coord), y_coords.append(y_coord), counts.append(UMIs)

# Convert to DF for sorting -important to get highest UMIs last
whole_experiment_plotting_df = pd.DataFrame(index = SBs, data = {'x_coords': x_coords, 'y_coords': y_coords, 'counts': counts}).sort_values('counts', ascending=True)

# Plot whole experiment UMIs
fig, ax = plt.subplots()
ax.set_title(f'Whole Experiment UMIs')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(whole_experiment_plotting_df['x_coords'], whole_experiment_plotting_df['y_coords'], s = 10, c = whole_experiment_plotting_df['counts'], alpha = 0.5, cmap = 'Blues')
plt.colorbar(sc)

# Save to disk
fig.savefig('gel_2_deep/whole experiment_visualization.png')

c = np.array(whole_experiment_plotting_df['counts'])
total_matched_SB_UMIs = sum(c)
normalized_c = c / total_matched_SB_UMIs
whole_experiment_plotting_df['counts_normalized_whole_experiment'] = normalized_c

# Plot whole experiment UMIs normalied - linear
fig, ax = plt.subplots()
ax.set_title(f'Whole Experiment UMIs - Linear Normalized')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc_linear = ax.scatter(whole_experiment_plotting_df['x_coords'], whole_experiment_plotting_df['y_coords'], s = 10, c = whole_experiment_plotting_df['counts_normalized_whole_experiment'], alpha = 0.5, cmap = 'Blues')
fig.colorbar(sc_linear)

# Save to disk
fig.savefig('gel_2_deep/whole_experiment_normalized_linear_visualization.png')

In [ ]:
### CONFIDENT - BACKGROUND RESIDUAL SB VISUALIZATION ###

print(f'Plotting confident - nuclei normalized heatmap...\n')

# Get residual DF
residual_plotting_df = confident_plotting_df.merge(background_plotting_df, how = 'left', left_index = True, right_index=True).fillna(0)
residual_plotting_df['residual'] = residual_plotting_df['counts_normalized_confident'] - residual_plotting_df['counts_normalized_background']
residual_plotting_df.sort_values('residual', inplace=True)

# Plot residual UMIs
fig, ax = plt.subplots(constrained_layout = True)
ax.set_title(f'Residual UMIs: Normalized Residual of Confident (<20k) and Background (>40k)')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(residual_plotting_df['x_coords_x'], residual_plotting_df['y_coords_x'], s = 10, c = residual_plotting_df['residual'], vmin = -0.00005, vmax = 0.00005, alpha = 0.5, cmap = 'RdBu')
plt.colorbar(sc)

# Save to disk
fig.savefig('gel_2_deep/residual_UMI_visualization.png')

# HOW TO NORMALIZE WITH NEGATIVE NUMBERS???

In [ ]:
### WHOLE EXPERIMENT - AMBIENT RESIDUAL SB VISUALIZATION ###

print(f'Plotting whole experiment - background normalized heatmap...\n')

# Get residual DF
residual_plotting_df = whole_experiment_plotting_df.merge(background_plotting_df, how = 'left', left_index = True, right_index=True).fillna(0)
residual_plotting_df['residual'] = residual_plotting_df['counts_normalized_whole_experiment'] - residual_plotting_df['counts_normalized_background']
residual_plotting_df.sort_values('residual', inplace=True)

# Plot residual UMIs
fig, ax = plt.subplots(constrained_layout = True)
ax.set_title(f'Residual UMIs: Normalized Residual of Whole Experiment and Background (40k - 100k)')
ax.set_xlim(0,6500)
ax.set_ylim(0,6500)
sc = ax.scatter(residual_plotting_df['x_coords_x'], residual_plotting_df['y_coords_x'], s = 10, c = residual_plotting_df['residual'], vmin = -0.00005, vmax = 0.00005, alpha = 0.5, cmap = 'RdBu', edgecolor='none', linewidths=0.5)
plt.colorbar(sc)

# Save to disk
fig.savefig('gel_2_deep/whole_experiment_residual_UMI_visualization.png')

# HOW TO NORMALIZE WITH NEGATIVE NUMBERS???

# c = np.array(confident_plotting_df['counts'])
# total_matched_SB_UMIs = sum(c)
# normalized_c = c / total_matched_SB_UMIs

In [ ]:
def plot_CBs(CBs_to_plot, CB_SB_mapping, coordinates, SBs_to_plot = None, normalized_ambient_background = None, title = None, save_file_path = None):
    # Get number of CBs to plot
    num_CBs = len(CBs_to_plot)

    # Ratio of signal SBs to ambient SBs
    SNRs = [0.01, 0.1, 0.5]
    if normalized_ambient_background == None:
        nrows = 2
    else:
        nrows = 2 + len(SNRs)

    # Keep track of total SB UMIs in droplet
    # Create figure to plot CBs on
    fig, axs = plt.subplots(nrows = nrows, ncols = num_CBs, figsize = (6 * num_CBs, 4 * nrows), constrained_layout = True, sharex = True, sharey = True)
    
    if title:
        fig.suptitle(title)

    # Iterate over CBs
    for col, CB in enumerate(CBs_to_plot):
        print(f'Plotting Cell Barcode: {CB}...')

        # Keep track of SBs in droplet
        droplet_SB_UMIs = 0

        # Get two axes, one for raw counts, one for normalized counts
        ax_raw = axs[0]
        ax_norm = axs[1]

        # Get SBs and initialize mapping lists
        SBs_in_CB = CB_SB_mapping[CB]
        x_coords, y_coords, counts, observed_SBs = [], [], [], []

        # Get UMI counts for SBs
        for SB, nUMIs in SBs_in_CB.items():
            if (SBs_to_plot is None) or (SB in SBs_to_plot):
                x_coord, y_coord = coordinates.loc[SB]
                nUMIs = int(nUMIs)
                droplet_SB_UMIs += nUMIs
                x_coords.append(x_coord), y_coords.append(y_coord), counts.append(nUMIs)
                observed_SBs.append(SB)

        # Normalize SB counts
        counts_normalized = np.array(counts) / droplet_SB_UMIs

        # Convert to DF for sorting
        plotting_df = pd.DataFrame(data = {'x_coords': x_coords, 'y_coords': y_coords, 'counts': counts, 'counts_normalized': counts_normalized})
        
        # Plot raw UMIs
        ax_raw.set_title(f'Cell Barcode: {CB}')
        ax_raw.set_xlim(0, 6500)
        ax_raw.set_ylim(0, 6500)
        plotting_df.sort_values('counts').plot.scatter(x = 'x_coords', y = 'y_coords', c = 'counts', ax = ax_raw, s = 5, alpha = 1, cmap = 'Blues')

        # Plot normalized UMIs
        ax_norm.set_title(f'Normalized: Total SB UMIs: {droplet_SB_UMIs}')
        ax_norm.set_xlim(0, 6500)
        ax_norm.set_ylim(0, 6500)
        plotting_df.sort_values('counts_normalized').plot.scatter(x = 'x_coords', y = 'y_coords', c = 'counts_normalized', ax = ax_norm, s = 5, alpha = 1, cmap = 'Blues')

        if normalized_ambient_background != None:
            total_observed_ambient_density = sum([normalized_ambient_background[SB] for SB in observed_SBs])
            adjusted_observed_ambient_background = {SB: normalized_ambient_background[SB] / total_observed_ambient_density for SB in set(observed_SBs)}

            for i, SNR in enumerate(SNRs):
                ax_adj = axs[2+i, col]
                SNR_col = f'counts_adjusted_{SNR}'
                counts_adjusted = [max([0, UMIs - (1-SNR) * droplet_SB_UMIs * adjusted_observed_ambient_background[observed_SBs[i]]]) for i, UMIs in enumerate(counts)]
                plotting_df[SNR_col] = counts_adjusted
                
                # Plot adjusted UMIs
                ax_adj.set_title(f'Adjusted: Raw SB UMIs minus ambient SB UMIs with SNR {SNR}')
                ax_adj.set_xlim(0, 6500)
                ax_adj.set_ylim(0, 6500)
                filtered_plotting_df = plotting_df[plotting_df[SNR_col] > 1]
                filtered_plotting_df.sort_values(SNR_col).plot.scatter(x = 'x_coords', y = 'y_coords', c = SNR_col, ax = ax_adj, s = 20, alpha = 1, cmap = 'Blues', linewidths=0.5)
        
            display(plotting_df.sort_values('counts', ascending = False).head(10))
    # Save
    if save_file_path:
        fig.savefig(save_file_path)

In [ ]:
def plot_coordinate_hist(CBs_to_plot, CB_SB_mapping, coordinates, SBs_to_plot = None, title = None, save_file_path = None):
    nrows = 2
    num_CBs = len(CBs_to_plot)

    fig, axs = plt.subplots(nrows = nrows, ncols = num_CBs, figsize = (6 * num_CBs, 4 * nrows), constrained_layout = True, sharex = True, sharey = True)
    
    if title:
        fig.suptitle(title)

    # Iterate over CBs
    for col, CB in enumerate(CBs_to_plot):
        print(f'Plotting Cell Barcode: {CB}...')

        # Keep track of SBs in droplet
        x_coord_appearances, y_coord_appearances = [], []
        missed_SBs = 0

        # Get two axes, one for x coordinate counts, one for y coordinate
        ax_x = axs[0,col]
        ax_y = axs[1,col]

        # Get SBs and initialize mapping lists
        SBs_in_CB = CB_SB_mapping[CB]

        # Get UMI counts for SBs
        for SB, nUMIs in SBs_in_CB.items():
            if (SBs_to_plot is None) or (SB in SBs_to_plot):
                try:
                    xcoord, ycoord = coordinates.loc[SB]
                    x_coord_appearances += [xcoord] * nUMIs
                    y_coord_appearances += [ycoord] * nUMIs
                except:
                    missed_SBs += 1

        print(f'Missed {missed_SBs} SBs for CB {CB}')
        # Plot x coordinate UMIs
        ax_x.set_title(f'X Coordinates')
        ax_x.hist(x_coord_appearances, bins = 100, range = (3500, 5500))

        # Plot x coordinate UMIs
        ax_y.set_title(f'Y Coordinates')
        ax_y.hist(y_coord_appearances, bins = 100, range = (1500, 3500))

    # Save
    if save_file_path:
        fig.savefig(save_file_path)
    
    plt.show()

In [ ]:
plot_coordinate_hist([random_c18_CBs[0], random_c18_CBs[1], random_c18_CBs[2]], spatial_SBs_in_CBs, SB_coordinates, SBs_to_plot=top_SBs, title = 'Cluster 18 CBs', save_file_path = '../BarcodeBender/official_data/gel_2_deep_cluster_18_top_SB_coordinate_distribution.png')

In [ ]:
# Calculate normalized UMI counts for ambient background
total_ambient_UMIs = 0

for SB, UMIs in ambient_background.items():
    total_ambient_UMIs += UMIs

normalized_ambient_background = {SB: UMIs / total_ambient_UMIs for SB, UMIs in ambient_background.items()}

In [ ]:
random_c18_CBs = random.choices(cluster_18_CBs, k = 3)
for i, CB in enumerate(cluster_18_CB_SB_counts_AD.obs_names):
    plot_CBs([CB], spatial_SBs_in_CBs, SB_coordinates, title = f'Cluster 18 CB {i}', save_file_path = f'../BarcodeBender/official_data/CBs_gel_2_deep_cluster_18/gel_2_deep_CB_{i}.png')
    plot_CBs([CB], spatial_SBs_in_CBs, SB_coordinates, SBs_to_plot=top_SBs, title = f'Cluster 18 CB {i} with top SBs', save_file_path = f'../BarcodeBender/official_data/CBs_gel_2_deep_cluster_18_top_SBs/gel_2_deep_CB_{i}_top_SBs.png')

In [ ]:
cluster_18_CB_SB_counts_AD

In [ ]:
### PLOT SOME CBS ###

# Number of CBs to plot
num_CBs = 3

# Get only CellRanger-called CBs
called_combined_UMI_df = combined_UMI_df.sort_values('Total GEX UMIs', ascending = False).head(min_ambient)

# Get random and top called CBs
# random_called_CBs = random.choices(sorted(called_combined_UMI_df.index), k = num_CBs)
# top_called_CBs = called_combined_UMI_df.sort_values('Total SB UMIs',ascending = False).head(num_CBs).index

# Plot CBs
plot_CBs(random_called_CBs, spatial_SBs_in_CBs, SB_coordinates, normalized_ambient_background, title = 'Random Called CBs', save_file_path = 'gel_2_deep/random_called_CBs_visualization.png')
# plot_CBs(top_called_CBs, spatial_SBs_in_CBs, SB_coordinates, title = 'Called CBs - Most SB UMIs', save_file_path = 'gel_2_deep/top_called_CBs_visualization.png')


In [ ]:
CB_SB_counts_AD = anndata.read_h5ad('gel_2_deep/CB_SB_counts.h5ad')

In [ ]:
ambient_CB_SB_counts_AD = CB_SB_counts_AD[ambient_CBs, :]
sc.tl.pca(ambient_CB_SB_counts_AD)
sc.pl.pca(ambient_CB_SB_counts_AD)
sc.pl.pca_variance_ratio(ambient_CB_SB_counts_AD, log=False)

called_CB_SB_counts_AD = CB_SB_counts_AD[list(CellRanger_called_CBs), :]
sc.pp.neighbors(called_CB_SB_counts_AD)
sc.tl.umap(called_CB_SB_counts_AD)
sc.pl.umap(called_CB_SB_counts_AD)